<a href="https://colab.research.google.com/github/IndraniMandal/New-Revisions/blob/main/Multi_Layered_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Constructing a basic ANN/MLP

Let's build some MLPs.  A fundamental problem with MLP design is the sheer number of design possibilities of these models. The MLP classisfier as part of the sklearn package has 23 (!) tunable paramaters. The good news is that all of these parameters except for the architectural parameters and the maximum number of training iterations have good default values. For the architectural parameters, a good starting point is an MLP  with a single hidden layer where the number of nodes in the hidden layer is computed as follows:

$ \#\mbox{nodes} = 2 \times \#\mbox{vars}$

That is the number of hidden nodes is twice the number of independent variables in the training data.  For the maximum number of training iterations we simply choose a very large value, e.g. 10,000. Let's try this using the pancreatic cancer dataset. Do note that it has already been cleaned and filled in.

In [ ]:
def classification_confint(acc, n):
    '''
    Compute the 95% confidence interval for a classification problem.
      acc -- classification accuracy
      n   -- number of observations used to compute the accuracy
    Returns a tuple (lb,ub)
    '''
    import math
    interval = 1.96*math.sqrt(acc*(1-acc)/n)
    lb = max(0, acc - interval)
    ub = min(1.0, acc + interval)
    return (lb,ub)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
df_filled = pd.read_csv("https://raw.githubusercontent.com/dpereztorres/2023WinterInternship/main/pancreas_df_filled.csv")
df_filled.head()

,patient_cohort,sample_origin,age,sex,diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,Cohort1,BPTB,33,F,1,11.7,1.83222,0.893219,52.94884,654.282174,1262.000
1,Cohort1,BPTB,81,F,1,9.0,0.97266,2.037585,94.46703,209.488250,228.407
2,Cohort2,BPTB,51,M,1,7.0,0.78039,0.145589,102.36600,461.141000,0.000
3,Cohort2,BPTB,61,M,1,8.0,0.70122,0.002805,60.57900,142.950000,0.000
4,Cohort2,BPTB,62,M,1,9.0,0.21489,0.000860,65.54000,41.088000,0.000


Determine the shape of the dataset as to not guess

In [ ]:
X  = df_filled.drop(['patient_cohort','sample_origin', 'sex', 'diagnosis'],axis=1)
y = df_filled['diagnosis']

print("Shape: {}".format(X.shape))

Shape: (590, 7)


Start with a pre-set hidden layer double the size of the amount of columns, with 'relu' being the default activation parameter.

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(14,), activation = 'relu', max_iter=10000, random_state=1)

model.fit(X, y)
predict_y = model.predict(X)
acc = accuracy_score(y, predict_y)
lb, ub = classification_confint(acc, X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy: 0.62 (0.58, 0.66)


#ReLU Function

<img src = "https://drive.google.com/uc?id=1OYCr7-vpzi3wvJbpXbmyMCGvOaAWZrCN" width=400>
<br>
<font size = '5'>
$ReLU(x) = max(x,0)$

#Sigmoid/Logistic Function

<img src = "https://drive.google.com/uc?id=15ofuYhq4AZJBdL05zYPy1hxDcspDsBe6" width=400>
<br>
<font size = '5'>
$sigmoid(x) = \frac{1}{1+e^{-x}}$

#Tahn Function

<img src = "https://drive.google.com/uc?id=1lVB9moXEN9M_v0It9W7FTHhRJwbJV0vc" width=400>
<br>
<font size = '5'>
$tanh(x) = \frac{1-e^{-2x}}{1+e^{-2x}}$


Images source and further details: [5.1 Mulitlayer Perceptrons](https://d2l.ai/chapter_multilayer-perceptrons/mlp.html#hidden-layers)

Now have three different layer sizes, each a multiple of the column amount, x2 x4 and x8. We also choose between three different activation parameters.

In [ ]:
model = MLPClassifier(max_iter=10000, random_state=1)
param_grid = {
    'hidden_layer_sizes': 
      [ 
      (14,), (28,), (56,),
      ],
    'activation' : ['logistic', 'tanh', 'relu']   
}

grid = GridSearchCV(model, param_grid, cv=3) 
grid.fit(X, y)
print("Grid Search: best parameters: {}".format(grid.best_params_))

best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

In [ ]:
model = MLPClassifier(max_iter=10000, random_state=1)
param_grid = {
    'hidden_layer_sizes': 
      [ 
      (14,), (28,), (60,),           # 1 hidden layer 
      (14,14), (14, 28), (14, 56),    # 2 hidden layers
      (28, 14), (28,28), (28, 56),    
      (56, 14), (56, 28), (56, 56)
      ],
    'activation' : ['logistic', 'tanh', 'relu']   
}

grid = GridSearchCV(model, param_grid, cv=5) 
grid.fit(X, y)
print("Grid Search: best parameters: {}".format(grid.best_params_))

best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

NameError: ignored

In [ ]:
model = MLPClassifier(max_iter=10000, random_state=1)
param_grid = {
    'hidden_layer_sizes': 
      [ 
      (14,), (28,), (56,),            
      (14,14), (14, 28), (14, 56),    
      (28, 14), (28,28), (28, 56),    
      (56, 14), (56, 28), (56, 56)
      ],
    'activation' : ['tanh', 'relu']   
}

grid = GridSearchCV(model, param_grid, cv=3) 
grid.fit(X, y)
print("Grid Search: best parameters: {}".format(grid.best_params_))

best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

Grid Search: best parameters: {'activation': 'tanh', 'hidden_layer_sizes': (56, 56)}
Accuracy: 0.90 (0.87,0.92)


In [ ]:
model = MLPClassifier(max_iter=10000, random_state=1)
param_grid = {
    'hidden_layer_sizes': 
      [ 
      (14,), (28,), (56,),            
      (14,14), (14, 28), (14, 56),    
      (28, 14), (28,28), (28, 56),    
      (56, 14), (56, 28), (56, 56)
      ],
    'activation' : ['logistic','relu']   
}

grid = GridSearchCV(model, param_grid, cv=3) 
grid.fit(X, y)
print("Grid Search: best parameters: {}".format(grid.best_params_))

best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

Grid Search: best parameters: {'activation': 'logistic', 'hidden_layer_sizes': (56,)}
Accuracy: 0.87 (0.84,0.89)


In [ ]:
model = MLPClassifier(max_iter=10000, random_state=1)
param_grid = {
    'hidden_layer_sizes': 
      [ 
      (14,), (28,), (56,),            
      (14,14), (14, 28), (14, 56),    
      (28, 14), (28,28), (28, 56),    
      (56, 14), (56, 28), (56, 56)
      ],
    'activation' : ['logistic', 'tanh']   
}

grid = GridSearchCV(model, param_grid, cv=3) 
grid.fit(X, y)
print("Grid Search: best parameters: {}".format(grid.best_params_))

best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

Grid Search: best parameters: {'activation': 'logistic', 'hidden_layer_sizes': (56,)}
Accuracy: 0.87 (0.84,0.89)
